In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
tf.keras.backend.clear_session()

In [6]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 60
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [7]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),...,Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,5.020927e+07,...,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,247.134
1,2003-2,0.756708,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,5.018273e+07,...,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,305.708
2,2003-3,0.756879,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,5.016725e+07,...,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,284.168
3,2003-4,0.757050,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,5.016559e+07,...,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,286.594
4,2003-5,0.757222,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,5.011916e+07,...,8.245980e+10,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,559.950


In [8]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.968698,-1.575817,-2.202855,-0.752424,-1.806910,-1.971018,-3.111816,-2.777792,1.874594,-1.361502,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-1.943192,-1.560482,-2.182122,-0.759292,-1.786501,-1.919572,-3.024730,-2.713279,1.869312,-1.359746,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-1.917685,-1.548550,-2.161328,-0.766160,-1.766091,-1.868126,-2.937643,-2.648766,1.866230,-1.358722,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-1.892178,-1.548848,-2.156390,-0.773028,-1.745682,-1.816680,-2.850557,-2.584252,1.865900,-1.358612,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-1.866672,-1.550544,-2.136466,-0.779896,-1.725272,-1.765234,-2.763471,-2.519739,1.856658,-1.352530,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.081278,1.208028,1.070020,0.646276,0.920674,-0.608959,-0.023342,0.468241,-0.624727,1.575950,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.064113,1.256499,1.058717,0.633557,0.902641,-0.600196,-0.045810,0.445578,-0.548742,1.764167,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.046949,1.305916,1.081210,0.620837,0.884609,-0.591433,-0.068278,0.422914,-0.439805,1.909130,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.029785,1.356297,1.085207,0.608118,0.866576,-0.582670,-0.090746,0.400250,-0.327203,2.040785,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      305.708
1      284.168
2      286.594
3      312.455
4      299.651
        ...   
235    612.428
236    579.831
237    559.950
238    559.950
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [10]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
60,-0.617077,-0.757795,-0.238292,-1.028493,-0.822444,0.927636,0.275859,-0.396951,-0.253335,0.485766,...,-0.907428,-1.023454,-0.537241,-0.615167,0.042292,0.488894,-0.717137,-0.601292,-0.315948,0.240813
61,-0.598696,-0.727075,-0.209865,-1.025680,-0.813752,0.973169,0.285191,-0.394134,-0.296485,0.526179,...,-0.899853,-1.024085,-0.479316,-0.525870,0.011943,-0.072763,-0.714911,-0.580436,-0.286095,0.246566
62,-0.580315,-0.690455,-0.153086,-1.022868,-0.805061,1.018702,0.294523,-0.391316,-0.338941,0.566133,...,-0.892039,-1.024307,-0.466312,-0.510476,-0.019339,-0.316940,-0.694168,-0.559580,-0.256243,0.315327
63,-0.561934,-0.651003,-0.114447,-1.020055,-0.796370,1.064234,0.303855,-0.388499,-0.380655,0.605591,...,-0.883981,-1.024104,-0.438778,-0.537853,-0.059264,0.180777,-0.700209,-0.538723,-0.226390,0.356189
64,-0.543553,-0.607147,-0.062262,-1.017242,-0.787679,1.109767,0.313188,-0.385682,-0.421583,0.644515,...,-0.875677,-1.023461,-0.457839,-0.446299,-0.104671,-0.212484,-0.660717,-0.517867,-0.196537,0.524165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.334554,0.717612,-0.030629,1.239700,1.136971,-0.986304,0.021589,0.677638,-0.685481,-1.113174,...,1.272681,1.521503,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.337445,0.715300,-0.017187,1.195852,1.142243,-0.994112,0.041435,0.688796,-0.757697,-1.166440,...,1.288866,1.512161,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.340335,0.716462,-0.003286,1.152004,1.147515,-1.001919,0.061280,0.699954,-0.844729,-1.204659,...,1.304998,1.500026,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.343226,0.715461,0.000561,1.108156,1.152788,-1.009727,0.081125,0.711112,-0.914820,-1.250199,...,1.332157,1.486995,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

60     402.782000
61     429.874000
62     484.463000
63     498.487000
64     506.647000
          ...    
163    411.310457
164    406.131343
165    400.952229
166    395.773114
167    390.594000
Name: Minas Gerais - Consumo de Cimento (t), Length: 108, dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
168,1.349007,0.715008,0.019780,1.020461,1.163332,-1.025342,0.120816,0.733427,-1.028195,-1.328800,...,1.364625,1.460212,-1.032458,0.157997,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117
169,1.343125,0.715570,0.029139,1.001834,1.162557,-1.011264,0.132877,0.734965,-1.102985,-1.327468,...,1.371257,1.445627,-1.077854,0.365531,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274
170,1.337243,0.716709,0.052997,0.983208,1.161781,-0.997185,0.144939,0.736502,-1.168851,-1.298530,...,1.375328,1.430008,-1.036724,0.269739,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341
171,1.331361,0.718059,0.071070,0.964582,1.161006,-0.983107,0.157000,0.738040,-1.188592,-1.273097,...,1.381257,1.413730,-1.013400,0.165991,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301
172,1.325480,0.719639,0.140932,0.945955,1.160230,-0.969029,0.169061,0.739577,-1.180922,-1.191250,...,1.364882,1.396980,-1.059228,0.257434,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284
173,1.319598,0.721466,0.186483,0.927329,1.159455,-0.954950,0.181122,0.741115,-1.124581,-1.154162,...,1.369812,1.379116,-1.170032,0.159711,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348
174,1.313716,0.723499,0.258333,0.908702,1.158679,-0.940872,0.193184,0.742652,-1.064934,-1.137769,...,1.376543,1.360141,-1.115621,0.034630,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659
175,1.307834,0.725580,0.325035,0.890076,1.157904,-0.926794,0.205245,0.744190,-1.111858,-1.110088,...,1.384496,1.341361,-1.182159,-0.009939,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184
176,1.301952,0.727713,0.348449,0.871450,1.157128,-0.912715,0.217306,0.745727,-1.162547,-1.073320,...,1.391358,1.323058,-1.270623,-0.061769,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173
177,1.296071,0.729905,0.378109,0.852823,1.156353,-0.898637,0.229368,0.747265,-1.165964,-1.015391,...,1.399740,1.303196,-1.321931,-0.169304,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168    338.767
169    419.962
170    387.800
171    391.122
172    409.739
173    413.460
174    497.019
175    476.478
176    421.395
177    413.641
178    338.033
179    431.028
180    305.105
181    402.406
182    419.419
183    336.054
184    568.395
185    541.488
186    537.851
187    526.265
188    559.293
189    466.646
190    416.049
191    501.489
192    460.699
193    445.583
194    509.271
195    532.602
196    500.905
197    588.495
198    589.349
199    543.982
200    614.074
201    557.419
202    412.582
203    468.600
Name: Minas Gerais - Consumo de Cimento (t), dtype: float64

In [15]:
def validation_splitter(df, div_factor, add_factor):
    split_factor = len(df)//div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop_index.append(pos + add_factor)
        positions_to_drop.append(pos)
    df_result = df.drop(positions_to_drop_index) 
    df_droped = df.iloc[positions_to_drop]
    return df_result, df_droped

In [26]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 10, 60)
    target,target_val = validation_splitter(train_target, 10, 60)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    reshaped_train = train.to_numpy().reshape(train.shape[0], 1, train.shape[1])
    reshaped_train_val = train_val.to_numpy().reshape(train_val.shape[0], 1, train_val.shape[1])
    reshaped_target = target.values.reshape(-1, 1)
    reshaped_target_val = target_val.values.reshape(-1, 1)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(592, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(74, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(reshaped_train, 
                        reshaped_target, 
                        epochs=10000,
                        validation_data=(reshaped_train_val,
                                         reshaped_target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[4172569970, 3599686509, 3920130888, 2552476511, 1425640108, 3276917987, 4284132321, 1312173310, 2268939567, 3889967492, 2955927522, 2963506008, 2341373771, 2011606329, 1961759604, 1419615160, 153286107, 664032288, 1055783338, 3015576033, 3332334470, 4091015039, 4070371767, 3971202772, 1707817332]


Step: 0 ___________________________________________
val_loss: 825.2784423828125
winner_seed: 4172569970


Step: 1 ___________________________________________
val_loss: 714.4898681640625
winner_seed: 3599686509


Step: 2 ___________________________________________
val_loss: 724.7664184570312


Step: 3 ___________________________________________
val_loss: 542.8739624023438
winner_seed: 2552476511


Step: 4 ___________________________________________
val_loss: 1023.8904418945312


Step: 5 ___________________________________________
val_loss: 1837.265625


Step: 6 ___________________________________________
val_loss: 651.1392822265625


Step: 7 ___________________________________________
val_lo

In [29]:
trained_model, history = lstm_model(train_input, 
                                    train_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 127ms/step - loss: 211507.1094 - val_loss: 8455.4082
Epoch 2/10000
4/4 [==============================] - 0s 7ms/step - loss: 5508.6401 - val_loss: 9761.9355
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 5677.9961 - val_loss: 9237.4736
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 6034.9932 - val_loss: 12312.5742
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 7053.6914 - val_loss: 8853.8438
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 5662.9912 - val_loss: 8299.5596
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 5613.3672 - val_loss: 9042.0488
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 5786.5972 - val_loss: 9975.4307
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 7535.6069 - val_loss: 9939.0684
Epoch 10/10000
4/4 [==============================

4/4 [==============================] - 0s 7ms/step - loss: 4168.9180 - val_loss: 2348.3057
Epoch 79/10000
4/4 [==============================] - 0s 7ms/step - loss: 3228.2314 - val_loss: 4690.8813
Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 4142.9902 - val_loss: 1648.1321
Epoch 81/10000
4/4 [==============================] - 0s 7ms/step - loss: 3530.9919 - val_loss: 2693.1675
Epoch 82/10000
4/4 [==============================] - 0s 8ms/step - loss: 3606.6118 - val_loss: 2596.0911
Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 3295.8154 - val_loss: 5580.6694
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 3797.2654 - val_loss: 3715.7141
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 3199.0022 - val_loss: 1885.5256
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 3216.5291 - val_loss: 1186.4017
Epoch 87/10000
4/4 [==============================] - 0s 7ms/

Epoch 155/10000
4/4 [==============================] - 0s 7ms/step - loss: 2806.6899 - val_loss: 1242.7590
Epoch 156/10000
4/4 [==============================] - 0s 7ms/step - loss: 2638.2178 - val_loss: 975.2577
Epoch 157/10000
4/4 [==============================] - 0s 7ms/step - loss: 2546.3450 - val_loss: 5333.3076
Epoch 158/10000
4/4 [==============================] - 0s 7ms/step - loss: 3128.2820 - val_loss: 1804.9792
Epoch 159/10000
4/4 [==============================] - 0s 7ms/step - loss: 2612.9905 - val_loss: 1823.0238
Epoch 160/10000
4/4 [==============================] - 0s 7ms/step - loss: 2530.9607 - val_loss: 5544.8022
Epoch 161/10000
4/4 [==============================] - 0s 7ms/step - loss: 4079.6699 - val_loss: 5358.0264
Epoch 162/10000
4/4 [==============================] - 0s 7ms/step - loss: 3624.5137 - val_loss: 4505.3276
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 3082.6919 - val_loss: 1997.7878
Epoch 164/10000
4/4 [=================

4/4 [==============================] - 0s 7ms/step - loss: 3219.7483 - val_loss: 1116.5725
Epoch 232/10000
4/4 [==============================] - 0s 7ms/step - loss: 3099.1846 - val_loss: 2318.0930
Epoch 233/10000
4/4 [==============================] - 0s 7ms/step - loss: 3094.7188 - val_loss: 1223.1077
Epoch 234/10000
4/4 [==============================] - 0s 8ms/step - loss: 2562.8018 - val_loss: 845.8954
Epoch 235/10000
4/4 [==============================] - 0s 7ms/step - loss: 2899.5527 - val_loss: 734.3156
Epoch 236/10000
4/4 [==============================] - 0s 8ms/step - loss: 2773.0024 - val_loss: 892.2344
Epoch 237/10000
4/4 [==============================] - 0s 8ms/step - loss: 3504.7310 - val_loss: 1814.2488
Epoch 238/10000
4/4 [==============================] - 0s 7ms/step - loss: 3111.4700 - val_loss: 1139.9377
Epoch 239/10000
4/4 [==============================] - 0s 8ms/step - loss: 2685.7690 - val_loss: 1718.0662
Epoch 240/10000
4/4 [==============================] - 0

Epoch 308/10000
4/4 [==============================] - 0s 8ms/step - loss: 2542.5967 - val_loss: 2100.3901
Epoch 309/10000
4/4 [==============================] - 0s 7ms/step - loss: 2730.5071 - val_loss: 1561.2117
Epoch 310/10000
4/4 [==============================] - 0s 7ms/step - loss: 2473.6208 - val_loss: 2714.3474
Epoch 311/10000
4/4 [==============================] - 0s 7ms/step - loss: 3002.7500 - val_loss: 3323.8035
Epoch 312/10000
4/4 [==============================] - 0s 7ms/step - loss: 2781.8667 - val_loss: 2726.6309
Epoch 313/10000
4/4 [==============================] - 0s 7ms/step - loss: 2438.4370 - val_loss: 1643.1953
Epoch 314/10000
4/4 [==============================] - 0s 7ms/step - loss: 2466.6846 - val_loss: 4826.0156
Epoch 315/10000
4/4 [==============================] - 0s 7ms/step - loss: 3048.9197 - val_loss: 1454.8319
Epoch 316/10000
4/4 [==============================] - 0s 7ms/step - loss: 2182.9590 - val_loss: 1265.8025
Epoch 317/10000
4/4 [================

...

4/4 [==============================] - 0s 7ms/step - loss: 3389.6804 - val_loss: 3935.7739
Epoch 767/10000
4/4 [==============================] - 0s 7ms/step - loss: 2863.5093 - val_loss: 1351.7849
Epoch 768/10000
4/4 [==============================] - 0s 7ms/step - loss: 2867.1116 - val_loss: 2966.0522
Epoch 769/10000
4/4 [==============================] - 0s 7ms/step - loss: 2509.1379 - val_loss: 1305.1854
Epoch 770/10000
4/4 [==============================] - 0s 7ms/step - loss: 2860.1958 - val_loss: 2156.2886
Epoch 771/10000
4/4 [==============================] - 0s 7ms/step - loss: 3201.2156 - val_loss: 1669.2894
Epoch 772/10000
4/4 [==============================] - 0s 7ms/step - loss: 3187.9219 - val_loss: 1485.8870
Epoch 773/10000
4/4 [==============================] - 0s 7ms/step - loss: 2773.4209 - val_loss: 1151.9980
Epoch 774/10000
4/4 [==============================] - 0s 8ms/step - loss: 3337.8586 - val_loss: 1011.0106
Epoch 775/10000
4/4 [==============================] 

Epoch 843/10000
4/4 [==============================] - 0s 8ms/step - loss: 3628.5613 - val_loss: 11021.7656
Epoch 844/10000
4/4 [==============================] - 0s 7ms/step - loss: 6572.3257 - val_loss: 2731.1372
Epoch 845/10000
4/4 [==============================] - 0s 7ms/step - loss: 3572.8274 - val_loss: 2146.8357
Epoch 846/10000
4/4 [==============================] - 0s 7ms/step - loss: 2860.9312 - val_loss: 1031.6460
Epoch 847/10000
4/4 [==============================] - 0s 7ms/step - loss: 2910.0581 - val_loss: 1024.7030
Epoch 848/10000
4/4 [==============================] - 0s 7ms/step - loss: 2910.3848 - val_loss: 1468.1781
Epoch 849/10000
4/4 [==============================] - 0s 7ms/step - loss: 2957.3354 - val_loss: 2912.6187
Epoch 850/10000
4/4 [==============================] - 0s 7ms/step - loss: 3220.5618 - val_loss: 3133.8130
Epoch 851/10000
4/4 [==============================] - 0s 7ms/step - loss: 3472.7612 - val_loss: 1586.3875
Epoch 852/10000
4/4 [===============

4/4 [==============================] - 0s 7ms/step - loss: 3534.5435 - val_loss: 967.2347
Epoch 920/10000
4/4 [==============================] - 0s 7ms/step - loss: 2994.9722 - val_loss: 940.0417
Epoch 921/10000
4/4 [==============================] - 0s 7ms/step - loss: 3295.2258 - val_loss: 2334.5488
Epoch 922/10000
4/4 [==============================] - 0s 7ms/step - loss: 3563.9468 - val_loss: 1476.0793
Epoch 923/10000
4/4 [==============================] - 0s 7ms/step - loss: 3428.2437 - val_loss: 1480.1660
Epoch 924/10000
4/4 [==============================] - 0s 7ms/step - loss: 4013.0105 - val_loss: 1818.9977
Epoch 925/10000
4/4 [==============================] - 0s 7ms/step - loss: 3672.4873 - val_loss: 1264.7615
Epoch 926/10000
4/4 [==============================] - 0s 7ms/step - loss: 2820.6780 - val_loss: 1277.0287
Epoch 927/10000
4/4 [==============================] - 0s 7ms/step - loss: 2564.0886 - val_loss: 2952.1155
Epoch 928/10000
4/4 [==============================] - 

4/4 [==============================] - 0s 7ms/step - loss: 2611.0076 - val_loss: 1229.5801
Epoch 996/10000
4/4 [==============================] - 0s 8ms/step - loss: 2518.9346 - val_loss: 1252.2213
Epoch 997/10000
4/4 [==============================] - 0s 7ms/step - loss: 2591.0581 - val_loss: 1178.5872
Epoch 998/10000
4/4 [==============================] - 0s 7ms/step - loss: 3106.2056 - val_loss: 1166.8633
Epoch 999/10000
4/4 [==============================] - 0s 7ms/step - loss: 2566.7490 - val_loss: 1850.3113
Epoch 1000/10000
4/4 [==============================] - 0s 7ms/step - loss: 2588.4648 - val_loss: 1281.0916
Epoch 1001/10000
4/4 [==============================] - 0s 7ms/step - loss: 2454.9675 - val_loss: 1332.7875
Epoch 1002/10000
4/4 [==============================] - 0s 7ms/step - loss: 2399.5408 - val_loss: 3103.1079
Epoch 1003/10000
4/4 [==============================] - 0s 8ms/step - loss: 3265.0740 - val_loss: 1399.4358
Epoch 1004/10000
4/4 [===========================

4/4 [==============================] - 0s 8ms/step - loss: 3199.6541 - val_loss: 1104.9436
Epoch 1072/10000
4/4 [==============================] - 0s 7ms/step - loss: 2965.2590 - val_loss: 7209.7393
Epoch 1073/10000
4/4 [==============================] - 0s 7ms/step - loss: 4036.3987 - val_loss: 4603.7744
Epoch 1074/10000
4/4 [==============================] - 0s 7ms/step - loss: 3603.8501 - val_loss: 1737.5442
Epoch 1075/10000
4/4 [==============================] - 0s 7ms/step - loss: 3348.1152 - val_loss: 2568.6047
Epoch 1076/10000
4/4 [==============================] - 0s 8ms/step - loss: 3031.6177 - val_loss: 1654.5437
Epoch 1077/10000
4/4 [==============================] - 0s 7ms/step - loss: 3006.0000 - val_loss: 1448.4944
Epoch 1078/10000
4/4 [==============================] - 0s 8ms/step - loss: 3168.0847 - val_loss: 4824.9429
Epoch 1079/10000
4/4 [==============================] - 0s 7ms/step - loss: 4063.8347 - val_loss: 1939.7585
Epoch 1080/10000
4/4 [=======================

In [38]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input.iloc[i:i+1]
        prediction = model.predict(input_test.to_numpy().reshape(1, 1, input_test.shape[1]))
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [39]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        test_input, 
                                        test_target, 168)

1/1 [==============================] - 0s 16ms/step
Month-1 - Error: [[[72.15463]]]
1/1 [==============================] - 0s 15ms/step
Month-2 - Error: [[[11.018555]]]
1/1 [==============================] - 0s 14ms/step
Month-3 - Error: [[[20.593536]]]
1/1 [==============================] - 0s 15ms/step
Month-4 - Error: [[[17.2659]]]
1/1 [==============================] - 0s 15ms/step
Month-5 - Error: [[[1.3519897]]]
1/1 [==============================] - 0s 14ms/step
Month-6 - Error: [[[5.086151]]]
1/1 [==============================] - 0s 14ms/step
Month-7 - Error: [[[88.63452]]]
1/1 [==============================] - 0s 15ms/step
Month-8 - Error: [[[70.372955]]]
1/1 [==============================] - 0s 15ms/step
Month-9 - Error: [[[13.979858]]]
1/1 [==============================] - 0s 15ms/step
Month-10 - Error: [[[7.695404]]]
1/1 [==============================] - 0s 14ms/step
Month-11 - Error: [[[67.799805]]]
1/1 [==============================] - 0s 14ms/step
Month-12 - Error:

In [40]:
display(mae)
display(mape)

77.882935

0.15707265

In [41]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input.iloc[i:i+1]
        prediction = model.predict(input_test.to_numpy().reshape(1, 1, input_test.shape[1]))
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [42]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[[4892.8296]]] - Target[4938.444]| =  Error: [[[45.614258]]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[[4831.899]]] - Target[5580.459999999999]| =  Error: [[[748.56104]]]
1/1 [==============================] - 0s 14ms/step
Ano-11: |Prediction[[[4886.285]]] - Target[6223.561000000001]| =  Error: [[[1337.2759]]]


[array([[[45.614258]]], dtype=float32),
 array([[[748.56104]]], dtype=float32),
 array([[[1337.2759]]], dtype=float32)]

710.4837

0.119416445